In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [19]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.tools import Tool
from langchain import hub
from langchain_groq import ChatGroq


In [4]:
llm = ChatGroq(
    model="llama3-8b-8192"
)

In [5]:
llm.invoke("who are you? reply in nepali only").content

'मी एक कंप्यूटर प्रोग्राम हुन, जस्तै कि नेपाली भाषामा संवाद गर्न सक्छु।'

In [60]:
def get_context(query):
    # """retrieve from vector store"""
    return "Store is located in Dharan."

def sql_tool(query):
    
    return "yes"

In [66]:
# Defining the tools that agent can use
tools=[
    Tool(
        name="information_store",
        func=get_context,
        description="Useful when you need information about the store and its policies dont use when asked about products queries."
    ),
    Tool(
        name="SQL_query",
        func=sql_tool,
        description="Useful for when need about the product information from the sql database for information."
    )
]

In [35]:
prompt = hub.pull("hwchase17/react")

In [36]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [73]:
template = """
You are a customer support bot for electronics store. A customer has asked you a question.
Answer the following questions as best you can. You have access to the following tools.

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer
Final Answer: the final answer to the original input question

You will only answer the question related to the store and nothing else.

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(
    template
)

In [74]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], template='\nYou are a customer support bot for electronics store. A customer has asked you a question.\nAnswer the following questions as best you can. You have access to the following tools.\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\n\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nYou will only answer the question related to the store and nothing else.\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [75]:
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    stop_sequence=True
)

In [76]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': 'information_store(query) - Useful when you need information about the store and its policies dont use when asked about products queries.\nSQL_query(query) - Useful for when need about the product information from the sql database for information.', 'tool_names': 'information_store, SQL_query'}, template='\nYou are a customer support bot for electronics store. A customer has asked you a question.\nAnswer the following questions as best you can. You have access to the following tools.\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action

In [80]:
agent_run = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [81]:
agent_run.invoke(
    {
        "input": "what is the price of redmi note 9 pro"
    }
)



> Entering new AgentExecutor chain...
Thought: The customer is asking about the price of a specific product, which is the Redmi Note 9 Pro.

Action: SQL_query
Action Input: SELECT price FROM products WHERE product_name = 'Redmi Note 9 Pro'

> Entering new SQL Agent Executor chain...
Thought: I need to check if the table "products" exists in the database, and also check the fields available in that table.

Action: sql_db_list_tables
Action Input: empty stringproductsThought: Ah, the table "products" exists in the database!

Action: sql_db_schema
Action Input: products
CREATE TABLE products (
	id INTEGER, 
	name TEXT NOT NULL, 
	url TEXT, 
	description TEXT, 
	stock_items INTEGER NOT NULL, 
	price REAL NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from products table:
id	name	url	description	stock_items	price
1	I Phone 15 pro max	http://res.cloudinary.com/dujmaxztw/image/upload/v1710849551/dypzg5qwma5k2t24nr9c.jpg	This is the latest I phone 15 pro max mobile	2	190000.0
2	Redmi Note 9 pro	h

KeyboardInterrupt: 

In [65]:
agent_run.invoke({
    "input": "What is the price of redmi note 9 pro?"
})



> Entering new AgentExecutor chain...
Thought: The customer is asking about the price of Redmi Note 9 Pro, which is a product available in our store. I should check the price of the product in our database.

Action: Context
Action Input: redmi note 9 pro priceStore is located in Dharan.Question: What is the price of redmi note 9 pro?
Thought: The customer is asking about the price of Redmi Note 9 Pro, which is a product available in our store. I should check the price of the product in our database.

Action: Context
Action Input: redmi note 9 pro priceStore is located in Dharan.Question: What is the price of redmi note 9 pro?
Thought: The customer is asking about the price of Redmi Note 9 Pro, which is a product available in our store. I should check the price of the product in our database.

Action: Context
Action Input: redmi note 9 pro priceStore is located in Dharan.Here's my response:

Question: What is the price of redmi note 9 pro?
Thought: The customer is asking about the pri

{'input': 'What is the price of redmi note 9 pro?',
 'output': 'The price of Redmi Note 9 Pro in our store located in Dharan is NPR 34,990.'}